In [1]:
from textblob import TextBlob
import pandas as pd
import sqlalchemy
#import pyodbc

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('inreality').config('spark.jars.packages','com.microsoft.azure:spark-mssql-connector_2.12:1.2.0').getOrCreate()

server_name = "jdbc:sqlserver://inreality.database.windows.net:1433"
database_name = "data"
url = server_name + ";" + "databaseName=" + database_name + ";"

table_name = "dbo.review"
username = ""
password = "" # Please specify password here

jdbcDF = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password)\
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
        .load()
jdbcDF.show()

22/10/07 16:01:00 WARN Utils: Your hostname, TamdeMacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.100 instead (on interface en0)
22/10/07 16:01:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/opt/homebrew/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/danie/.ivy2/cache
The jars for the packages stored in: /Users/danie/.ivy2/jars
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5b2887ed-f0c2-46b5-b49e-730b6550c55f;1.0
	confs: [default]
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 90ms :: artifacts dl 6ms
	:: modules in use:
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |

22/10/07 16:01:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+--------------------+--------------------+--------------------+--------------------+-------+--------------+
|        prodcut_name|        reviewerName|             summary|          reviewText|overall|unixReviewTime|
+--------------------+--------------------+--------------------+--------------------+-------+--------------+
|        Parasite Eve|ACEMAN1 "HERE TO ...|A FANTASTIC HORRO...|Have you ever won...|    5.0|    03-16-2008|
|     Resident Evil 2|        B. E Jackson|Want to see blood...|Now THIS is how y...|    5.0|    09-07-2005|
|       Mario Kart 64|M. King "The supe...|       Mario kart 64|This mario kart c...|    4.0|    06-24-2009|
|         Silent Hill|            Kevin M.|                Hehe|Even as a 31 year...|    4.0|    01-25-2014|
|Nintendo 64 Syste...|                Nate|Best Multiplayer ...|Remember getting ...|    5.0|    01-09-2013|
|    Mortal Kombat II|David "Black Wido...|       It has Begun.|MK 2 Rocks, this ...|    5.0|    05-10-2006|
|   Super Mario Bro

In [4]:

from pyspark.sql.types import *
from pyspark.sql.functions import col
short_df = jdbcDF.select('prodcut_name','reviewText','summary')

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag_sents
 #   return list_words
panda_df = short_df.toPandas()
yes = panda_df['reviewText'].tolist()
list = [str(i) for i in yes]
tagged_texts = pos_tag_sents(map(word_tokenize, list))

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle[0m

  Searched in:
    - '/Users/danie/nltk_data'
    - '/opt/homebrew/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/nltk_data'
    - '/opt/homebrew/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/share/nltk_data'
    - '/opt/homebrew/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
panda_df['pos'] = tagged_texts
def finding_adj(text):
    adj_list = [word_pair[0] for word_pair in text if word_pair[1] == 'JJ' and TextBlob(word_pair[0]).sentiment[0] != 0 and TextBlob(word_pair[0]).sentiment[1] != 0]
    return adj_list
panda_df['adj'] = panda_df['pos'].apply(finding_adj)

NameError: name 'tagged_texts' is not defined

In [ ]:
import re
adj_df = panda_df[['prodcut_name','adj']]
# We want to the prouct name to be a particular column, so we need to use index=False to avoid turing them into column
adj_groupby_df = adj_df.groupby('prodcut_name',as_index=False).sum()

def word_cleansing(wordlist):
    new_wordlist = [re.sub("[+.-]",'',word.lower()) for word in wordlist]
    return new_wordlist
adj_groupby_df['adj'] = adj_groupby_df['adj'].apply(word_cleansing)
display(adj_groupby_df)

In [ ]:
from collections import OrderedDict
def reduce_map_on_counting_word_appearance(wordlist):
    order_dict = {}
    for word in wordlist:
        if word not in order_dict.keys():
            order_dict[word] = 1 
        else:
            order_dict[word] += 1
    count_desc_dict = sorted(order_dict.items(),key=lambda x:x[1],reverse=True)
    return count_desc_dict[:10]

def getting_word(text):
    return text[0]
def getting_number(text):
    return text[1]

adj_groupby_df['adj_count'] = adj_groupby_df['adj'].apply(reduce_map_on_counting_word_appearance)
final_adj_group_df = adj_groupby_df[['prodcut_name','adj_count']]
final_adj_explode_df = final_adj_group_df.explode('adj_count')
final_adj_explode_df['word'] = final_adj_explode_df['adj_count'].apply(getting_word)
final_adj_explode_df['appearance'] = final_adj_explode_df['adj_count'].apply(getting_number)
final_adj_df = final_adj_explode_df.drop(columns=['adj_count'],axis=1)
final_adj_df.head(10)

In [ ]:
adj_df_spark = spark.createDataFrame(final_adj_df)
adj_df_table_name = 'dbo.game_adj_top_5'
adj_df_spark.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", adj_df_table_name) \
    .option("user", username) \
    .option("password", password) \
    .save()

In [ ]:
def get_ploarity(text):
    blob = TextBlob(text)
    plority = blob.sentiment[0]
    return plority 
spark_df = short_df.toPandas()
spark_df['score'] = spark_df['reviewText'].apply(get_ploarity)
group_df = spark_df.groupby('prodcut_name',as_index=False).mean().sort_values(by='score', ascending=True)

In [ ]:
group_df_spark = spark.createDataFrame(group_df)
group_df_table_name = 'dbo.product_polarity_score'
group_df_spark.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", group_df_table_name) \
    .option("user", username) \
    .option("password", password) \
    .save()